In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.exposure import match_histograms
from skimage import color
from google.colab.patches import cv2_imshow

In [ ]:
def gloablmatching(source, target,num_of_samples):
  #displaying the source and target images
  f, axArr = plt.subplots(nrows=2,ncols=3,figsize=(10,10))
  plt.sca(axArr[0,0])
  plt.imshow(source)
  plt.title('Source Image')
  
  plt.sca(axArr[0,1])
  plt.imshow(target,cmap='gray')
  plt.title('Target Image')

  #converting the RBG source image to LAB form
  LABmatrix = color.rgb2lab(source/255)

  #plotting the histogram of the l values of the source image
  plt.sca(axArr[1,0])
  plt.hist(LABmatrix[:,:,0])
  plt.title('Histogram: Source Image')

  #plotting the histogram of the intensities of the target image
  plt.sca(axArr[1,1])
  plt.hist(target[:,:])
  plt.title('Histogram: Target Image')


  #histogram matching 
  source_lvals = match_histograms(LABmatrix[:,:,0],target)
  LABmatrix[:,:,0] = source_lvals
  plt.sca(axArr[1,2])
  plt.hist(LABmatrix[:,:,0])
  plt.title('Histogram Matching: Source Image')

  #divide the image into 200 parts
  # dividing the x axis in nx=10 parts
  # and y axis in ny = 20 parts
  # take a random pixel from each part
  #The array Z stores the 200 random pixels (x,y)
  #the array Z also stores the comparison metrix: 0.5*sd + 0.5l(x,y)
  src = source
  np.random.seed(42)
  nx = 20 #number of divisions along x
  ny = np.int(num_of_samples/20) #number of divisions along y
  x = LABmatrix.shape[0]
  y = LABmatrix.shape[1]
  divx = x/nx
  divy = y/ny
  count = 0
  Z = np.zeros((num_of_samples,3))
  for i in range(nx):
    for j in range(ny):
      Z[count][0] = int(min(np.random.randint(i*divx,(i+1)*divx,1)+2,x-2))
      Z[count][1] = int(min(np.random.randint(j*divy,(j+1)*divy,1)+2,y-2))
      xcord = int(Z[count][0])
      ycord = int(Z[count][1])
      #Marking the selected random point
      src = cv2.circle(src, (ycord,xcord),radius=0,color=(255,0,0),thickness=2)
      #the comparison metric
      Z[count][2] = 0.5*np.std(np.ravel(LABmatrix[xcord-2:xcord+2,ycord-2:ycord+2,0])) + 0.5*LABmatrix[xcord,ycord,0]
      count = count + 1

  plt.sca(axArr[0,2])
  plt.imshow(src)
  plt.title('Sampled Points')
  plt.show()

  #in the target image go through each pixel and compute the nbd stats with the available neighbors in 5x5 neighborhood
  # compare it with the 200 values of the source image and choose that value for which the mod of the difference is min
  # transfer a and b channels for the min value
  coloured_img = np.zeros((target.shape[0],target.shape[1],3))
  x = target.shape[0]
  y = target.shape[1]
  for i in range(x):
    for j in range(y):
      l = [] 
      for i1 in range(-2,3):
        for j1 in range(-2,3):
          #take the avaliable pixels for calculating the nbs stats
          if(i+i1>=0 and j+j1>=0 and i+i1<x and j+j1<y):
            l.append(target[int(i+i1),int(j+j1)])
      metric = np.std(l)*0.5 + target[i,j]*0.5
      indx = abs(Z[:,2] - metric).argmin()
      #the l value is between 0-100, hence scale it
      coloured_img[i,j,0] = target[i,j]*(100/255)
      coloured_img[i,j,1] = LABmatrix[int(Z[indx][0]),int(Z[indx][1]),1]
      coloured_img[i,j,2] = LABmatrix[int(Z[indx][0]),int(Z[indx][1]),2] 

  
  #lab image to rgb img
  result = color.lab2rgb(coloured_img)
  return result

In [ ]:
source1 = cv2.imread('source1.png')
source1 = cv2.cvtColor(source1, cv2.COLOR_BGR2RGB)
target1 = cv2.imread('target1.png',cv2.IMREAD_GRAYSCALE)
result = gloablmatching(source1, target1,400)
plt.imshow(result)
plt.title('Final coloured image')

In [ ]:
#Only for plotting results
'''
f, axArr = plt.subplots(nrows=1,ncols=3,figsize=(10,3))
source1 = cv2.imread('source1.png')
source1 = cv2.cvtColor(source1, cv2.COLOR_BGR2RGB)
plt.sca(axArr[0])
plt.imshow(source1,aspect='auto')
plt.title('Source Image')
plt.sca(axArr[1])
target1 = cv2.imread('target1.png',cv2.IMREAD_GRAYSCALE)
plt.imshow(target1,cmap='gray',aspect='auto')
plt.title('Target Image')
plt.sca(axArr[2])
plt.imshow(result,aspect='auto')
plt.title('Final coloured image')
plt.show()
'''

In [ ]:
source2 = cv2.imread('source2.png')
source2 = cv2.cvtColor(source2, cv2.COLOR_BGR2RGB)
target2 = cv2.imread('target2.png',cv2.IMREAD_GRAYSCALE)
result = gloablmatching(source2, target2,400)
plt.imshow(result)
plt.title('Final coloured image')

In [ ]:
#Only for plotting results
'''
f, axArr = plt.subplots(nrows=1,ncols=3,figsize=(10,3))
source2 = cv2.imread('source2.png')
source2 = cv2.cvtColor(source2, cv2.COLOR_BGR2RGB)
plt.sca(axArr[0])
plt.imshow(source2,aspect='auto')
plt.title('Source Image')
plt.sca(axArr[1])
target2 = cv2.imread('target2.png',cv2.IMREAD_GRAYSCALE)
plt.imshow(target2,cmap='gray',aspect='auto')
plt.title('Target Image')
plt.sca(axArr[2])
plt.imshow(result,aspect='auto')
plt.title('Final coloured image')
plt.show()
'''

In [ ]:
source3 = cv2.imread('source3.png')
source3 = cv2.cvtColor(source3, cv2.COLOR_BGR2RGB)
target3 = cv2.imread('target3.png',cv2.IMREAD_GRAYSCALE)
result = gloablmatching(source3, target3,400)
plt.imshow(result)
plt.title('Final coloured image')

In [ ]:
#Only for plotting results
'''
f, axArr = plt.subplots(nrows=1,ncols=3,figsize=(10,3))
source3 = cv2.imread('source3.png')
source3 = cv2.cvtColor(source3, cv2.COLOR_BGR2RGB)
plt.sca(axArr[0])
plt.imshow(source3,aspect='auto')
plt.title('Source Image')
plt.sca(axArr[1])
target3 = cv2.imread('target3.png',cv2.IMREAD_GRAYSCALE)
plt.imshow(target3,cmap='gray',aspect='auto')
plt.title('Target Image')
plt.sca(axArr[2])
plt.imshow(result,aspect='auto')
plt.title('Final coloured image')
plt.show()
'''

In [ ]:
source4 = plt.imread('source4.jpeg')
target4 = cv2.imread('target4.jpeg',cv2.IMREAD_GRAYSCALE)
result = gloablmatching(source4, target4,400)
plt.imshow(result)
plt.title('Final coloured image')

In [ ]:
#Only for plotting results
'''
f, axArr = plt.subplots(nrows=1,ncols=3,figsize=(10,3))
source4 = cv2.imread('source4.jpeg')
source4 = cv2.cvtColor(source4, cv2.COLOR_BGR2RGB)
plt.sca(axArr[0])
plt.imshow(source4,aspect='auto')
plt.title('Source Image')
plt.sca(axArr[1])
target4 = cv2.imread('target4.jpeg',cv2.IMREAD_GRAYSCALE)
plt.imshow(target4,cmap='gray',aspect='auto')
plt.title('Target Image')
plt.sca(axArr[2])
plt.imshow(result,aspect='auto')
plt.title('Final coloured image')
plt.show()
'''